In [60]:
import pulp
import pandas as pd
import numpy as np

In [61]:
PREFERENCES_FILE = "./preferences.csv"
ROOMS_FILE = "./rooms.csv"


In [62]:
def read_rooms(rooms_file: str) -> np.ndarray:
    return pd.read_csv(rooms_file, index_col=0).astype(int).to_numpy().flatten()

In [63]:
rooms = read_rooms(ROOMS_FILE)
NUM_ROOMS = rooms.shape[0]

In [64]:
def read_preferences(preferences_file: str) -> np.ndarray:
    return pd.read_csv(preferences_file, index_col=0).astype(int).to_numpy()

In [65]:
preferences = read_preferences(PREFERENCES_FILE)
NUM_RESIDENTS = preferences.shape[0]
NUM_PREFERENCE_CATEGORIES = preferences.shape[1]
preferences, NUM_RESIDENTS, NUM_PREFERENCE_CATEGORIES

(array([[2, 1, 1, 3, 1],
        [2, 1, 4, 3, 1],
        [1, 1, 4, 4, 2],
        [4, 1, 3, 4, 4],
        [3, 4, 1, 4, 4],
        [3, 3, 2, 2, 3],
        [3, 2, 4, 3, 2],
        [3, 3, 2, 1, 4],
        [2, 3, 3, 1, 4],
        [1, 4, 2, 2, 4],
        [2, 4, 4, 1, 1],
        [4, 1, 4, 2, 3],
        [3, 4, 2, 3, 1],
        [3, 2, 4, 4, 2],
        [3, 1, 3, 1, 3],
        [1, 4, 2, 2, 4]]),
 16,
 5)

In [89]:
# returns array where arr[i][j] represents the compatibility between two people, calculated as the sum of the absolute difference between the two vectors (might change to square root of the sum of the squares or something later!) or maybe the normalized dot product

def calculate_compatibility_array(preferences, num_residents=-1, num_preference_categories=-1):
    def calculate_compatibility(a_prefs, b_prefs):
        return np.sum(np.abs(a_prefs - b_prefs))

    if num_residents == -1:
        num_residents = preferences.shape[0]
    if num_preference_categories == -1:
        num_preference_categories = preferences.shape[1]

    compatibilities = []

    for a, a_prefs in enumerate(preferences):
        a_compatibilities = []
        for b, b_prefs in enumerate(preferences):
            if a == b:
                a_compatibilities.append(1000000)
            else:
                a_compatibilities.append(calculate_compatibility(a_prefs, b_prefs))
        compatibilities.append(a_compatibilities)
    
    return np.array(compatibilities)

compatibility_array = calculate_compatibility_array(preferences)
compatibility_array[9]
    
        

array([      9,      10,       9,       9,       5,       4,       9,
             4,       4, 1000000,       7,       9,       6,      10,
             8,       0])

In [47]:
# def read_schedule(schedule_file: str) -> list[list[int]]:
#     row_numbers = []
#     schedule = pd.read_csv("./schedule.csv",index_col=0).fillna(0).astype(int)
#     for col in schedule.columns:
#         row_numbers.append(schedule.loc[schedule[col] == 1].index.values.tolist())
#     return row_numbers


# SCHEDULE = read_schedule("./schedule.csv")
# SCHEDULE

In [90]:
problem = pulp.LpProblem('lpsolver', pulp.LpMinimize)

In [91]:
#create one variable for every resident-resident-room combo (NUM_RESIDENTS * NUM_RESIDENTS * NUM_ROOMS). we will also add a null resident to signify single rooms (but this is in v2). OR mAYBE make x_{ssj} signify that j is a single room with s assigned to it. So any single room should have one x_{ssj}, and any double room will have x_{stg} and x{tsg}
x = pulp.LpVariable.dicts('x', range(NUM_RESIDENTS * NUM_RESIDENTS * NUM_ROOMS), lowBound=0, upBound=1, cat='Integer')
NUM_RESIDENTS, NUM_ROOMS
print(len(x))

2048


In [92]:
preferences = read_preferences(PREFERENCES_FILE) # TODO: convert p into a 2D array rather than 1D (more readable)
compatibility_array = calculate_compatibility_array(preferences)
# problem.setObjective(sum([sum([p[NUM_EVENTS * i + j]*x[NUM_EVENTS * i + j] for j in range(NUM_EVENTS)])for i in range(NUM_STUDENTS)]))

problem.setObjective(
    sum([
        sum([
            sum([
                compatibility_array[s][t]*x[s * (NUM_ROOMS * NUM_RESIDENTS) +  t * (NUM_ROOMS) + j]
                for t in range(NUM_RESIDENTS)]) 
            for s in range(NUM_RESIDENTS)]) 
        for j in range(NUM_ROOMS)]))
problem

lpsolver:
MINIMIZE
1000000*x_0 + 1000000*x_1 + 3*x_10 + 5*x_100 + 8*x_1000 + 8*x_1001 + 8*x_1002 + 8*x_1003 + 8*x_1004 + 8*x_1005 + 8*x_1006 + 8*x_1007 + 4*x_1008 + 4*x_1009 + 5*x_101 + 4*x_1010 + 4*x_1011 + 4*x_1012 + 4*x_1013 + 4*x_1014 + 4*x_1015 + 4*x_1016 + 4*x_1017 + 4*x_1018 + 4*x_1019 + 5*x_102 + 4*x_1020 + 4*x_1021 + 4*x_1022 + 4*x_1023 + 9*x_1024 + 9*x_1025 + 9*x_1026 + 9*x_1027 + 9*x_1028 + 9*x_1029 + 5*x_103 + 9*x_1030 + 9*x_1031 + 8*x_1032 + 8*x_1033 + 8*x_1034 + 8*x_1035 + 8*x_1036 + 8*x_1037 + 8*x_1038 + 8*x_1039 + 7*x_104 + 9*x_1040 + 9*x_1041 + 9*x_1042 + 9*x_1043 + 9*x_1044 + 9*x_1045 + 9*x_1046 + 9*x_1047 + 7*x_1048 + 7*x_1049 + 7*x_105 + 7*x_1050 + 7*x_1051 + 7*x_1052 + 7*x_1053 + 7*x_1054 + 7*x_1055 + 7*x_1056 + 7*x_1057 + 7*x_1058 + 7*x_1059 + 7*x_106 + 7*x_1060 + 7*x_1061 + 7*x_1062 + 7*x_1063 + 4*x_1064 + 4*x_1065 + 4*x_1066 + 4*x_1067 + 4*x_1068 + 4*x_1069 + 7*x_107 + 4*x_1070 + 4*x_1071 + 7*x_1072 + 7*x_1073 + 7*x_1074 + 7*x_1075 + 7*x_1076 + 7*x_1077 + 7*x_10

In [93]:
# each resident can only be in 1 room (for each resident sum over all their variables)
for s in range(NUM_RESIDENTS):
    sum_expr = sum([sum([x[s * (NUM_ROOMS * NUM_RESIDENTS) +  t * (NUM_ROOMS) + j] for j in range(NUM_ROOMS)]) for t in range(NUM_RESIDENTS)])
    problem.addConstraint(sum_expr == 1)


In [94]:
# each room can only be assigned two residents
## (x_(stj)) and (x_(tsj)) to this room

for j in range(NUM_ROOMS):
    sum_expr = sum([sum([x[s * (NUM_ROOMS * NUM_RESIDENTS) +  t * (NUM_ROOMS) + j] for s in range(NUM_RESIDENTS)]) for t in range(NUM_RESIDENTS)])
    problem.addConstraint(sum_expr == 2)

In [95]:
# x_{stj} = x_{tsj}

for j in range(NUM_ROOMS):
    for s in range(NUM_RESIDENTS):
        for t in range(NUM_RESIDENTS):
            problem.addConstraint(x[s * (NUM_ROOMS * NUM_RESIDENTS) +  t * (NUM_ROOMS) + j] == x[t * (NUM_ROOMS * NUM_RESIDENTS) + s * (NUM_ROOMS) + j])

In [96]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kanav/opt/anaconda3/envs/scioly/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/g2/py91tm2d5bj465mlzrg0r1l40000gn/T/6c30ea4e17ef40878071f4ecc18eb5c6-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/g2/py91tm2d5bj465mlzrg0r1l40000gn/T/6c30ea4e17ef40878071f4ecc18eb5c6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2077 COLUMNS
At line 16270 RHS
At line 18343 BOUNDS
At line 20392 ENDATA
Problem MODEL has 2072 rows, 2048 columns and 7936 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 50 - 0.01 seconds
Cgl0004I processed model has 24 rows, 1088 columns (1088 integer (1088 of which binary)) and 3136 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 14 integers unsatisfied sum - 3.5
Cbc0038I Pass   1: suminf

1

In [97]:
values = [x[i].value() for i in range(len(x))]
all([value == 0.0 or value == 1.0 for value in values]), values

(True,
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [98]:
sum(values)

16.0

In [101]:
output = pd.DataFrame(np.array(values).reshape(15, 23)).astype(int)

(array([  96,  148,  268,  423,  543,  702,  874,  942, 1105, 1275, 1345,
        1525, 1536, 1714, 1885, 1995]),)

In [104]:
values = np.array(values)
reshaped = values.reshape((NUM_RESIDENTS, NUM_RESIDENTS, NUM_ROOMS))

In [102]:
values = np.array(values)
np.where(values == 1.0)

(array([  96,  148,  268,  423,  543,  702,  874,  942, 1105, 1275, 1345,
        1525, 1536, 1714, 1885, 1995]),)

In [135]:
 matchings = []
 dic = {}
 for s_idx, s in enumerate(reshaped):
    for t_idx, t in enumerate(s):
        if np.any(t):
            j_idx = np.where(t == 1.0)[0][0]
            if s_idx < t_idx:
                matchings.append((s_idx, t_idx, j_idx))
            # dic[j_idx] = (s_idx, t_idx)
pd.DataFrame(matchings).to_csv("./output.csv",index=False, header=["Roommate A", "Roommate B", "Room"])
matchings



[(0, 12, 0),
 (1, 2, 4),
 (3, 4, 7),
 (5, 7, 6),
 (6, 13, 2),
 (8, 10, 1),
 (9, 15, 3),
 (11, 14, 5)]

In [118]:
pd.DataFrame(matchings).to_csv("./output.csv")

In [165]:
event = 13
sum([x[NUM_EVENTS * student + event].value() for student in range(NUM_STUDENTS)])

2.0

In [166]:
slot = SCHEDULE[0]
student = 9
sum([x[NUM_EVENTS * student + event].value() for event in slot])

0.0